In [40]:
%pip install tensorflow


  Using cached numpy-2.1.3-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached MarkupSafe-3.0.2-cp312-cp312-win_amd64.whl.metadata (4.1 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
   ---------------------------------------- 0.0/376.0 MB ? eta -:--:--
   ---------------------------------------- 1.6/376.0 MB 9.3 MB/s eta 0:00:41
   ---------------------------------------- 3.9/376.0 MB 10.7 MB/s eta 0:00:35
    --------------------------------------- 6.0/376.0 MB 10.5 MB/s eta 0:00:36
    --------------------------------------- 8.4/376.0 MB 10.6 MB/s eta 0:00:35
   - -------------------------------------- 10.5/376.0 MB 10.6 MB/s eta 0:00:35
   - -------------------------------------- 12.6/376.0 MB 10.5 MB/s eta 0:00:35
   - -------------------------------------- 15.2/376.0 MB 10.6 MB/s eta 0:00:34
   - -------------------------------

  You can safely remove it manually.
  You can safely remove it manually.


In [2]:
%pip install cv2
%pip install numpy as np

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement cv2 (from versions: none)
ERROR: No matching distribution found for cv2


Note: you may need to restart the kernel to use updated packages.


ERROR: Ignored the following versions that require a different python version: 1.21.2 Requires-Python >=3.7,<3.11; 1.21.3 Requires-Python >=3.7,<3.11; 1.21.4 Requires-Python >=3.7,<3.11; 1.21.5 Requires-Python >=3.7,<3.11; 1.21.6 Requires-Python >=3.7,<3.11
ERROR: Could not find a version that satisfies the requirement as (from versions: none)
ERROR: No matching distribution found for as


In [41]:
import tensorflow as tf
from tensorflow import keras

In [2]:
%pip install opencv-python-headless

   ---------------------------------------- 0.0/38.9 MB ? eta -:--:--
   - -------------------------------------- 1.6/38.9 MB 9.4 MB/s eta 0:00:04
   ---- ----------------------------------- 4.2/38.9 MB 10.5 MB/s eta 0:00:04
   ------ --------------------------------- 6.6/38.9 MB 10.9 MB/s eta 0:00:03
   --------- ------------------------------ 8.9/38.9 MB 10.9 MB/s eta 0:00:03
   ----------- ---------------------------- 11.0/38.9 MB 10.7 MB/s eta 0:00:03
   ------------- -------------------------- 13.1/38.9 MB 10.8 MB/s eta 0:00:03
   --------------- ------------------------ 15.5/38.9 MB 10.7 MB/s eta 0:00:03
   ------------------ --------------------- 17.8/38.9 MB 10.8 MB/s eta 0:00:02
   -------------------- ------------------- 20.2/38.9 MB 10.9 MB/s eta 0:00:02
   ----------------------- ---------------- 22.5/38.9 MB 10.9 MB/s eta 0:00:02
   ------------------------- -------------- 24.9/38.9 MB 11.0 MB/s eta 0:00:02
   ---------------------------- ----------- 27.3/38.9 MB 11.1 MB/s

In [3]:
import cv2
import os
import numpy as np

In [4]:
project_path=r"C:\Users\Asus\Desktop\NeuroDoku"
sudoku_images=os.path.join(project_path,"sudoku_images")


In [5]:
sudoku_images

'C:\\Users\\Asus\\Desktop\\NeuroDoku\\sudoku_images'

In [10]:
data_list=os.listdir(sudoku_images)
print(data_list)
data_X_train=[]
data_X_test=[]
data_Y_train=[]
data_Y_test=[]
data_list_train_path=os.path.join(sudoku_images,"v2_train")

['mixed', 'mixed 2', 'outlines_sorted.csv', 'v1_test', 'v1_training', 'v2_test', 'v2_train']


In [18]:
data_list_train_path=os.path.join(sudoku_images,r"v2_train\v2_train")
data_list_test_path=os.path.join(sudoku_images,r"v2_test\v2_test")


In [ ]:
data_list_train=os.listdir(data_list_train_path)
data_list_test=os.listdir(data_list_test_path)


In [26]:
data_list_train

['image1.dat',
 'image1.jpg',
 'image10.dat',
 'image10.jpg',
 'image100.dat',
 'image100.jpg',
 'image1000.dat',
 'image1000.jpg',
 'image1001.dat',
 'image1001.jpg',
 'image1003.dat',
 'image1003.jpg',
 'image1004.dat',
 'image1004.jpg',
 'image1006.dat',
 'image1006.jpg',
 'image1007.dat',
 'image1007.jpg',
 'image1008.dat',
 'image1008.jpg',
 'image101.dat',
 'image101.jpg',
 'image1011.dat',
 'image1011.jpg',
 'image1014.dat',
 'image1014.jpg',
 'image102.dat',
 'image102.jpg',
 'image1020.dat',
 'image1020.jpg',
 'image1022.dat',
 'image1022.jpg',
 'image103.dat',
 'image103.jpg',
 'image1031.dat',
 'image1031.jpg',
 'image1036.dat',
 'image1036.jpg',
 'image1038.dat',
 'image1038.jpg',
 'image1039.dat',
 'image1039.jpg',
 'image104.dat',
 'image104.jpg',
 'image1040.dat',
 'image1040.jpg',
 'image1043.dat',
 'image1043.jpg',
 'image1045.dat',
 'image1045.jpg',
 'image1048.dat',
 'image1048.jpg',
 'image1049.dat',
 'image1049.jpg',
 'image105.dat',
 'image105.jpg',
 'image1050.da

In [22]:
prefix="image"
suffix=".jpg"
pre_len=len(prefix)
suff_len=len(suffix)

In [27]:

for image_name in data_list_train:
    if(image_name.lower().endswith(".jpg")):
        image_path=os.path.join(data_list_train_path,image_name)
        pic=cv2.imread(image_path)
        pic=cv2.resize(pic,(32,32))
        data_X_train.append(pic)
        label=int(image_name[pre_len:-suff_len])
        data_Y_train.append(label)




In [35]:
if len(data_X_train)==len(data_Y_train):
    print("Toatl Data Points :", len(data_Y_train))

Toatl Data Points : 160


In [36]:
for image_name in data_list_test:
    if(image_name.lower().endswith(".jpg")):
        image_path=os.path.join(data_list_test_path,image_name)
        pic=cv2.imread(image_path)
        pic=cv2.resize(pic,(32,32))
        data_X_test.append(pic)
        label=int(image_name[pre_len:-suff_len])
        data_Y_test.append(label)

In [37]:
if len(data_X_test)==len(data_Y_test):
    print("Toatl Data Points :", len(data_Y_test))

Toatl Data Points : 40


In [39]:
X_train=np.array(data_X_train)
X_test=np.array(data_X_test)
Y_train=np.array(data_Y_train)
Y_test=np.array(data_Y_test)

In [ ]:
train_y = to_categorical(Y_train, np.max(Y_train) + 1)